# 1. Import libraries

In [1]:
import pandas as pd
from DataCleaningPipeline import DataCleaner

In [2]:
superstore_df = pd.read_csv('superstore_raw.csv')

# 2. Check the data

In [3]:
print(superstore_df.head())

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal Code Region       Product ID         Cat

In [4]:
print(superstore_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

In [5]:
print(superstore_df['Country'].unique())

['United States']


Postal codes in USA are numerical. In this case, since the only country is United States, the data type can be changed from float to integer. For other countries it may not be advisable since that could lead to the same postal code from different places because it may have left zeros that are lost in the transition.

Change data types to:
- Date: Order Date, Ship Date
- Integer: Postal Code
- Category: Country, City, Region, Category, Sub-Category, Segment, Ship Mode, Customer ID, Product ID 

In [6]:
superstore_df.describe(include='all')

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
count,9800.000000,9800,9800,9800,9800,9800,9800,9800,9800,9800,9800,9789.000000,9800,9800,9800,9800,9800,9800.000000
unique,NaN,4922,1230,1326,4,793,793,3,1,529,49,NaN,4,1861,3,17,1849,NaN
top,NaN,CA-2018-100111,05/09/2017,26/09/2018,Standard Class,WB-21850,William Brown,Consumer,United States,New York City,California,NaN,West,OFF-PA-10001970,Office Supplies,Binders,Staple envelope,NaN
freq,NaN,14,38,34,5859,35,35,5101,9800,891,1946,NaN,3140,19,5909,1492,47,NaN
mean,4900.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55273.322403,NaN,NaN,NaN,NaN,NaN,230.769059
std,2829.160653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32041.223413,NaN,NaN,NaN,NaN,NaN,626.651875
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1040.000000,NaN,NaN,NaN,NaN,NaN,0.444000
25%,2450.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23223.000000,NaN,NaN,NaN,NaN,NaN,17.248000
50%,4900.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58103.000000,NaN,NaN,NaN,NaN,NaN,54.490000
75%,7350.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90008.000000,NaN,NaN,NaN,NaN,NaN,210.605000


Check the maximum lenght of object type of each column of th dataframe to determine PostgreSQL VARCHAR n size.

In [7]:
#Create a function to calculate the maximum length in each column from a dataframe
def max_lenght(df):
    # Receive a dataframe as input
    # Initialize an empty dictionary to store the maximum lengths
    lenghts = {}
    # Check if the column is of type object
    for columns in df.columns:
        if df[columns].dtype == 'object':
            # Calculate the maximum length of strings in the column
            lenghts[columns] = df[columns].astype(str).str.len().max()
        else:
            lenghts[columns] = None
    return lenghts

# Apply the function to each column in the dataframe
max_lenghts = max_lenght(superstore_df)
print(pd.Series(max_lenghts).sort_values(ascending=False))

Product Name     127.0
Customer Name     22.0
State             20.0
City              17.0
Product ID        15.0
Category          15.0
Ship Mode         14.0
Order ID          14.0
Country           13.0
Sub-Category      11.0
Segment           11.0
Order Date        10.0
Ship Date         10.0
Customer ID        8.0
Region             7.0
Row ID             NaN
Postal Code        NaN
Sales              NaN
dtype: float64


# 3. Define tables and primary/foreign keys (PK/FK)

Star Schema:

dim_customer
* customer_id (PK)
* customer_name
* segment

dim_product
* product_id (PK)
* product_name
* category
* sub_category

dim_region
* region_id (PK) - will be created
* country
* state
* city
* postal_code
* region

dim_date
* date_id (PK)
* year
* quarter
* month
* day
* week_day
* order_date
* ship_date

dim_ship_mode
* ship_mode_id
* ship_mode


fact_sales
* row_id (PK)
* customer_id (FK)
* product_id (FK)
* region_id (FK)
* date_id (FK)
* sales

# 4. Define relational model

4.1 customers → orders (1:N relationship)
A single customer can place multiple orders, but each order belongs to exactly one customer.

* customers.customer_id is the primary key
* orders.customer_id is a foreign key referencing it

Why this relationship exists:  
In the raw CSV, the same customer appears many times across different rows. Storing customers in a separate table prevents duplication of names and segments, and allows customer‑level analysis.

4.2 regions → orders (1:N relationship)
Each order is associated with one geographic location (city, state, region), but a region can contain many orders.
* regions.region_id is the primary key
* orders.region_id is a foreign key

Why this relationship exists:  
Location fields repeat heavily in the dataset. Creating a dedicated regions table avoids redundancy and enables geographic reporting (sales by state, region, etc.).

4.3 orders → order_items (1:N relationship)
An order can contain multiple products, but each order item belongs to exactly one order.
* orders.order_id is the primary key
* order_items.order_id is a foreign key

Why this relationship exists:  
In the CSV, each Order ID appears multiple times — once per product purchased. This relationship reflects that structure and allows detailed line‑item analysis.

# 5. Clean the dataframe

In [8]:
cleaner = DataCleaner() # Create an instance of the DataCleaner class
superstore_cleaned = cleaner.clean_data(superstore_df) # Clean the data using the clean_data method


Existing columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales']

Existing columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales']
Could not convert column 'Postal Code' to int. Reason: Cannot convert non-finite values (NA or inf) to integer
Column 'Country' converted to category.
Column 'City' converted to category.
Column 'Region' converted to category.
Column 'Category' converted to category.
Column 'Segment' converted to category.
Column 'Ship Mode' converted to category.
Column 'Customer ID' converted to category.
Column 'Product ID' converted to category.


It was not possible to convert post_code to integer since there were null-values. Will be updated after treatment.

## 5.1 Check the new data

In [9]:
print(superstore_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   row_id         9800 non-null   int64         
 1   order_id       9800 non-null   object        
 2   order_date     3959 non-null   datetime64[ns]
 3   ship_date      3815 non-null   datetime64[ns]
 4   ship_mode      9800 non-null   category      
 5   customer_id    9800 non-null   category      
 6   customer_name  9800 non-null   object        
 7   segment        9800 non-null   category      
 8   country        9800 non-null   category      
 9   city           9800 non-null   category      
 10  state          9800 non-null   object        
 11  postal_code    9789 non-null   float64       
 12  region         9800 non-null   category      
 13  product_id     9800 non-null   category      
 14  category       9800 non-null   category      
 15  sub_category   9800 n

In [10]:
print(superstore_cleaned.head())

   row_id        order_id order_date  ship_date       ship_mode customer_id  \
0       1  CA-2017-152156 2017-08-11 2017-11-11    Second Class    CG-12520   
1       2  CA-2017-152156 2017-08-11 2017-11-11    Second Class    CG-12520   
2       3  CA-2017-138688 2017-12-06        NaT    Second Class    DV-13045   
3       4  US-2016-108966 2016-11-10        NaT  Standard Class    SO-20335   
4       5  US-2016-108966 2016-11-10        NaT  Standard Class    SO-20335   

     customer_name    segment        country             city       state  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   postal_code region       product_id         category sub_ca

In [11]:
print(f"Null-values:\n{superstore_cleaned.isna().sum().sort_values(ascending=False)}")

Null-values:
ship_date        5985
order_date       5841
postal_code        11
row_id              0
order_id            0
ship_mode           0
customer_name       0
segment             0
country             0
customer_id         0
city                0
state               0
region              0
product_id          0
category            0
sub_category        0
product_name        0
sales               0
dtype: int64


How to deal with the null-values?
1. ship_date — 5985 missing values out of 9800 (~61%)
* The proportion of missing values is too high to justify dropping rows or removing the column.
* Replacing missing dates with "Unknown" is not appropriate because it would convert the entire column from datetime to object, breaking time-based operations.
* Since ship_date is not relevant to the current analysis, the safest approach is to keep the missing values as NaT.
* The column was already converted to datetime using clean_data(), so missing values are automatically represented as NaT.

Decision: Maintain missing values as NaT (Not a Time).

2. order_date — 5841 missing values out of 9800 (~60%)
* Similar to ship_date, the missing proportion is too large to remove rows or discard the column.
* Filling with "Unknown" would break the datetime type and introduce inconsistencies.
* The column is not required for the current analysis.
* After conversion with clean_data(), missing values are already represented as NaT.

Decision: Maintain missing values as NaT.

3. postal_code — 11 missing values out of 9800 (~0.1%)
* The missing proportion is very small (~1%), so imputation is safe.
* Although the column is not essential for the analysis, imputing missing values improves dataset completeness.
* The most reasonable approach is to retrieve the postal code from the corresponding State, assuming postal codes are consistent within each state.
* Alternatively, filling with "Unknown" would also be acceptable, but retrieving from State provides more accurate data.

Decision: Impute missing postal codes using the most common postal code within each State.

In [12]:
print(f"Duplicates:{superstore_cleaned.duplicated().sum()}")

Duplicates:0


In [20]:
#Create a function to update missing postal codes based on city
def update_postal_code(df, row):
     # If the postal code is missing
     if pd.isna(row["postal_code"]):
         city = row["city"] # Postal codes existing for the same city
         codes = df.loc[df["city"] == city, "postal_code"].dropna() # Get postal codes for the same city, excluding NaN values
         if len(codes) > 0: # If there are postal codes available for the city
            return codes.mode()[0] # Use the most common postal code (mode) 
         else: # Fallback if the city has no known postal codes 
            return 00000 
     else: # If the postal code already exists, keep it
        return row["postal_code"]

#Update missing postal codes in the cleaned dataframe
superstore_cleaned["postal_code"] = superstore_cleaned.apply(
    lambda row: update_postal_code(superstore_cleaned, row), axis=1
    ) # Apply the function row-wise

In [25]:
# Convert postal codes to integer type after replacing missing values
superstore_cleaned['postal_code'] = superstore_cleaned['postal_code'].astype(int)

In [26]:
superstore_cleaned.describe(include='all')

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
count,9800.000000,9800,3959,3815,9800,9800,9800,9800,9800,9800,9800,9800.000000,9800,9800,9800,9800,9800,9800.000000
unique,NaN,4922,NaN,NaN,4,793,793,3,1,529,49,NaN,4,1861,3,17,1849,NaN
top,NaN,CA-2018-100111,NaN,NaN,Standard Class,WB-21850,William Brown,Consumer,United States,New York City,California,NaN,West,OFF-PA-10001970,Office Supplies,Binders,Staple envelope,NaN
freq,NaN,14,NaN,NaN,5859,35,35,5101,9800,891,1946,NaN,3140,19,5909,1492,47,NaN
mean,4900.500000,NaN,2017-03-14 18:19:11.199798016,2017-04-09 17:04:02.516382720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55241.830612,NaN,NaN,NaN,NaN,NaN,230.769059
min,1.000000,NaN,2015-01-02 00:00:00,2015-01-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1040.000000,NaN,NaN,NaN,NaN,NaN,0.444000
25%,2450.750000,NaN,2016-04-05 00:00:00,2016-04-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23223.000000,NaN,NaN,NaN,NaN,NaN,17.248000
50%,4900.500000,NaN,2017-05-02 00:00:00,2017-06-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57551.000000,NaN,NaN,NaN,NaN,NaN,54.490000
75%,7350.250000,NaN,2018-03-07 00:00:00,2018-05-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90008.000000,NaN,NaN,NaN,NaN,NaN,210.605000
max,9800.000000,NaN,2018-12-11 00:00:00,2019-05-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99301.000000,NaN,NaN,NaN,NaN,NaN,22638.480000


# 6. Save the cleaned data as a csv file

In [27]:
superstore_cleaned.to_csv('superstore_cleaned.csv', index=False)

In [28]:
print(superstore_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   row_id         9800 non-null   int64         
 1   order_id       9800 non-null   object        
 2   order_date     3959 non-null   datetime64[ns]
 3   ship_date      3815 non-null   datetime64[ns]
 4   ship_mode      9800 non-null   category      
 5   customer_id    9800 non-null   category      
 6   customer_name  9800 non-null   object        
 7   segment        9800 non-null   category      
 8   country        9800 non-null   category      
 9   city           9800 non-null   category      
 10  state          9800 non-null   object        
 11  postal_code    9800 non-null   int64         
 12  region         9800 non-null   category      
 13  product_id     9800 non-null   category      
 14  category       9800 non-null   category      
 15  sub_category   9800 n

Postal code with no null-values and correct data type after update.